## Deepsensor Key classes.

* DataProcessor: Maps xarray and pandas data from their native units to a normalised and standardised format (and vice versa).

* TaskLoader: Slices and samples normalised xarray and pandas data to generate Task objects for training and inference.

* Task: Container for context and target data. Subclass of dict with additional methods for processing and summarising the data.

* DeepSensorModel: Base class for DeepSensor models, implementing a high-level .predict method for predicting straight to xarray/pandas in original coordinates and units.

* ConvNP: Convolutional neural process (ConvNP) model class (subclass of DeepSensorModel). Uses the neuralprocesses library. This is currently the only model provided by DeepSensor.

* Trainer: Class for training on Task objects using backpropagation and the Adam optimiser.

* AcquisitionFunction: Base class for active learning acquisition functions.

* GreedyAlgorithm: Greedy search algorithm for active learning.

In addition, a deepsensor.plot module provides useful plotting functions for visualising:

* Task context and target sets

* DeepSensorModel predictions.

* ConvNP internals (encoding and feature maps)

* GreedyAlgorithm active learning outputs.

In [1]:
pip install git+https://github.com/scott-hosking/get-station-data.git

  Cloning https://github.com/scott-hosking/get-station-data.git to /tmp/pip-req-build-lap9fuds
  Running command git clone --filter=blob:none --quiet https://github.com/scott-hosking/get-station-data.git /tmp/pip-req-build-lap9fuds
  Resolved https://github.com/scott-hosking/get-station-data.git to commit f7eaa50823ff75117b577b370e67d194c3d342a1
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [1]:
from deepsensor.data import DataProcessor
from deepsensor.data.sources import get_ghcnd_station_data, get_era5_reanalysis_data, get_earthenv_auxiliary_data, get_gldas_land_mask
import os

In [2]:
import logging
logging.captureWarnings(True)

import xarray as xr
import pandas as pd

# Using the same settings allows use to use pre-downloaded cached data
data_range = ("2015-06-25", "2015-06-30")
extent = "europe"
station_var_IDs = ["TAVG", "PRCP"]
era5_var_IDs = ["2m_temperature", "10m_u_component_of_wind", "10m_v_component_of_wind"]
aux_var_IDs = ["elevation", "tpi"]
cache_dir = "../../.datacache"

In [3]:
station_raw_df = get_ghcnd_station_data(station_var_IDs, extent, date_range=data_range, cache=True, cache_dir=cache_dir)
era5_raw_ds = get_era5_reanalysis_data(era5_var_IDs, extent, date_range=data_range, cache=True, cache_dir=cache_dir)
aux_raw_ds = get_earthenv_auxiliary_data(aux_var_IDs, extent, "1KM", cache=True, cache_dir=cache_dir)
land_mask_raw_ds = get_gldas_land_mask(extent, cache=True, cache_dir=cache_dir)

In [4]:
era5_raw_ds

<xarray.Dataset> Size: 2MB
Dimensions:                  (time: 6, lat: 141, lon: 221)
Coordinates:
  * lat                      (lat) float32 564B 70.0 69.75 69.5 ... 35.25 35.0
  * lon                      (lon) float32 884B -15.0 -14.75 ... 39.75 40.0
  * time                     (time) datetime64[ns] 48B 2015-06-25 ... 2015-06-30
Data variables:
    2m_temperature           (time, lat, lon) float32 748kB 274.7 ... 297.0
    10m_u_component_of_wind  (time, lat, lon) float32 748kB 6.309 ... 4.798
    10m_v_component_of_wind  (time, lat, lon) float32 748kB 3.221 ... -4.844
Attributes:
    valid_time_start:  1940-01-01
    last_updated:      2024-10-17 20:04:10.783634
    valid_time_stop:   2024-07-31

### Initialising a DataProcessor.
* defaults - time, x1, x2.

In [5]:
data_processor = DataProcessor(x1_name="lat", x2_name="lon")
print(data_processor)

DataProcessor with normalisation params:
{'coords': {'time': {'name': 'time'},
            'x1': {'map': None, 'name': 'lat'},
            'x2': {'map': None, 'name': 'lon'}}}


### Normalising data with DataProcessor.

In [6]:
era5_ds = data_processor(era5_raw_ds)
era5_ds

<xarray.Dataset> Size: 2MB
Dimensions:                  (time: 6, x1: 141, x2: 221)
Coordinates:
  * time                     (time) datetime64[ns] 48B 2015-06-25 ... 2015-06-30
  * x1                       (x1) float32 564B 0.6364 0.6318 ... 0.004545 0.0
  * x2                       (x2) float32 884B 0.0 0.004545 ... 0.9955 1.0
Data variables:
    2m_temperature           (time, x1, x2) float32 748kB -2.592 ... 1.452
    10m_u_component_of_wind  (time, x1, x2) float32 748kB 1.945 1.944 ... 1.423
    10m_v_component_of_wind  (time, x1, x2) float32 748kB 1.045 1.01 ... -1.412
Attributes:
    valid_time_start:  1940-01-01
    last_updated:      2024-10-17 20:04:10.783634
    valid_time_stop:   2024-07-31

In [7]:
station_raw_df

PRCP  TAVG
time       lat    lon     station                
2015-06-25 35.017 -1.450  AGM00060531   0.0  23.0
           35.100 -1.850  AGE00147716   0.0  23.4
           35.117  36.750 SYM00040030   NaN  25.4
           35.167  2.317  AGM00060514   0.0  25.9
           35.200 -0.617  AGM00060520   0.0  24.9
...                                     ...   ...
2015-06-30 45.933  7.700  ITM00016052   NaN   5.7
           38.367 -0.500  SPM00008359   0.0  27.6
           55.383  36.700 RSM00027611   0.0  17.2
           59.080  17.860 SWE00138750   0.0   NaN
           63.760  12.430 SWE00140158   0.6   NaN

[16928 rows x 2 columns]

In [15]:
station_df = data_processor(station_raw_df)
station_df

PRCP      TAVG
time       x1       x2       station                        
2015-06-25 0.000309 0.246364 AGM00060531 -0.278036  0.759107
           0.001818 0.239091 AGE00147716 -0.278036  0.836200
           0.002127 0.940909 SYM00040030       NaN  1.221668
           0.003036 0.314855 AGM00060514 -0.278036  1.318035
           0.003636 0.261509 AGM00060520 -0.278036  1.125301
...                                            ...       ...
2015-06-30 0.198782 0.412727 ITM00016052       NaN -2.575188
           0.061218 0.263636 SPM00008359 -0.278036  1.645682
           0.370600 0.940000 RSM00027611 -0.278036 -0.358749
           0.437818 0.597455 SWE00138750 -0.278036       NaN
           0.522909 0.498727 SWE00140158 -0.134218       NaN

[16928 rows x 2 columns]

### Can also process multiple variables in one Dataprocessor call.
* min_max - scale the data in a range [-1,1].

In [8]:
aux_raw_ds

<xarray.Dataset> Size: 222MB
Dimensions:    (lon: 6600, lat: 4200)
Coordinates:
  * lon        (lon) float64 53kB -15.0 -14.99 -14.98 ... 39.98 39.99 40.0
  * lat        (lat) float64 34kB 70.0 69.99 69.98 69.97 ... 35.02 35.01 35.0
Data variables:
    elevation  (lat, lon) float32 111MB 0.0 0.0 0.0 0.0 ... 261.6 261.5 260.6
    tpi        (lat, lon) float32 111MB 0.0 0.0 0.0 ... 0.03906 0.07812 0.1641

In [9]:
land_mask_raw_ds

<xarray.DataArray 'GLDAS_mask' (lat: 140, lon: 220)> Size: 123kB
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]], dtype=float32)
Coordinates:
  * lon      (lon) float32 880B -14.88 -14.62 -14.38 ... 39.38 39.62 39.88
  * lat      (lat) float32 560B 69.88 69.62 69.38 69.12 ... 35.62 35.38 35.12

In [10]:
aux_ds, land_mask_ds = data_processor([aux_raw_ds, land_mask_raw_ds], method="min_max")
aux_ds

<xarray.Dataset> Size: 222MB
Dimensions:    (x1: 4200, x2: 6600)
Coordinates:
  * x1         (x1) float64 34kB 0.6363 0.6361 0.636 ... 0.0002273 7.576e-05
  * x2         (x2) float64 53kB 7.576e-05 0.0002273 0.0003788 ... 0.9998 0.9999
Data variables:
    elevation  (x1, x2) float32 111MB -0.921 -0.921 -0.921 ... -0.8095 -0.8099
    tpi        (x1, x2) float32 111MB -0.09401 -0.09401 ... -0.09305 -0.09199

In [12]:
land_mask_ds

<xarray.DataArray 'GLDAS_mask' (x1: 140, x2: 220)> Size: 123kB
array([[-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       ...,
       [-1., -1., -1., ...,  1.,  1.,  1.],
       [-1., -1., -1., ...,  1.,  1.,  1.],
       [-1., -1., -1., ...,  1.,  1.,  1.]], dtype=float32)
Coordinates:
  * x1       (x1) float32 560B 0.6341 0.6295 0.625 ... 0.01136 0.006818 0.002273
  * x2       (x2) float32 880B 0.002273 0.006818 0.01136 ... 0.9932 0.9977

### DataProcessor configuration.
* Keeps track of the normalization parameters used to keep track of the data.

In [22]:
print(data_processor)

DataProcessor with normalisation params:
{'10m_u_component_of_wind': {'method': 'mean_std',
                             'params': {'mean': 0.6799039244651794,
                                        'std': 2.8934481143951416}},
 '10m_v_component_of_wind': {'method': 'mean_std',
                             'params': {'mean': -0.20978610217571259,
                                        'std': 3.282301187515259}},
 '2m_temperature': {'method': 'mean_std',
                    'params': {'mean': 288.9769287109375,
                               'std': 5.505463123321533}},
 'GLDAS_mask': {'method': 'min_max', 'params': {'max': 1.0, 'min': 0.0}},
 'PRCP': {'method': 'mean_std',
          'params': {'mean': 1.1599447860459278, 'std': 4.171918456167085}},
 'TAVG': {'method': 'mean_std',
          'params': {'mean': 19.0613726868119, 'std': 5.18850380936251}},
 'coords': {'time': {'name': 'time'},
            'x1': {'map': (35.0, 90.0), 'name': 'lat'},
            'x2': {'map': (-15.0, 40.0),

### Unnormalizing data.

In [13]:
era5_raw_ds_unnormalised = data_processor.unnormalise(era5_ds)
xr.testing.assert_allclose(era5_raw_ds, era5_raw_ds_unnormalised, atol=1e-6)
era5_raw_ds_unnormalised

<xarray.Dataset> Size: 2MB
Dimensions:                  (time: 6, lat: 141, lon: 221)
Coordinates:
  * time                     (time) datetime64[ns] 48B 2015-06-25 ... 2015-06-30
  * lat                      (lat) float32 564B 70.0 69.75 69.5 ... 35.25 35.0
  * lon                      (lon) float32 884B -15.0 -14.75 ... 39.75 40.0
Data variables:
    2m_temperature           (time, lat, lon) float32 748kB 274.7 ... 297.0
    10m_u_component_of_wind  (time, lat, lon) float32 748kB 6.309 ... 4.798
    10m_v_component_of_wind  (time, lat, lon) float32 748kB 3.221 ... -4.844
Attributes:
    valid_time_start:  1940-01-01
    last_updated:      2024-10-17 20:04:10.783634
    valid_time_stop:   2024-07-31

In [16]:
station_df_unnormalised = data_processor.unnormalise(station_df)
pd.testing.assert_frame_equal(station_raw_df, station_df_unnormalised)
station_df_unnormalised

PRCP  TAVG
time       lat    lon     station                
2015-06-25 35.017 -1.450  AGM00060531   0.0  23.0
           35.100 -1.850  AGE00147716   0.0  23.4
           35.117  36.750 SYM00040030   NaN  25.4
           35.167  2.317  AGM00060514   0.0  25.9
           35.200 -0.617  AGM00060520   0.0  24.9
...                                     ...   ...
2015-06-30 45.933  7.700  ITM00016052   NaN   5.7
           38.367 -0.500  SPM00008359   0.0  27.6
           55.383  36.700 RSM00027611   0.0  17.2
           59.080  17.860 SWE00138750   0.0   NaN
           63.760  12.430 SWE00140158   0.6   NaN

[16928 rows x 2 columns]

### Saving and loading a DataProcessor

In [16]:

data_processor.save("deepsensor_config/")
data_processor2 = DataProcessor("deepsensor_config/")
print(data_processor2)

DataProcessor with normalisation params:
{'10m_u_component_of_wind': {'method': 'mean_std',
                             'params': {'mean': 0.6799039244651794,
                                        'std': 2.8934481143951416}},
 '10m_v_component_of_wind': {'method': 'mean_std',
                             'params': {'mean': -0.20978610217571259,
                                        'std': 3.282301187515259}},
 '2m_temperature': {'method': 'mean_std',
                    'params': {'mean': 288.9769287109375,
                               'std': 5.505463123321533}},
 'GLDAS_mask': {'method': 'min_max', 'params': {'max': 1.0, 'min': 0.0}},
 'PRCP': {'method': 'mean_std',
          'params': {'mean': 1.1599447860459278, 'std': 4.171918456167085}},
 'TAVG': {'method': 'mean_std',
          'params': {'mean': 19.0613726868119, 'std': 5.18850380936251}},
 'coords': {'time': {'name': 'time'},
            'x1': {'map': (35.0, 90.0), 'name': 'lat'},
            'x2': {'map': (-15.0, 40.0),

### Normalization parameters over a subset of Data.

In [25]:
_ = data_processor(era5_raw_ds.sel(time=slice("2015-06-25", "2015-06-27")))
era5_ds = data_processor(era5_raw_ds)  # Will use the normalisation parameters computed above when called on the full dataset